In [18]:
import time 
import numpy as np
import ipyparallel as ipp
from cnn import CNN
import tensorflow as tf
from AggregatorFunc import snd_to_eng, wc_from_eng, global_update, bg_from_eng, belta_update, grad_update, delta_update, binary_search
from CnnFunc import cnn_parameter_initial,batch_gradient_collector,batch_parameter_collector,create_new_conv_layer,AdamOptimizer_Bing
from CnnFunc import conv_output_size,flatten_matrix,wrangle_matrix

In [19]:
c = ipp.Client()

In [20]:
dview = c[:]

In [21]:
c.ids

[0, 1]

In [22]:
import os
dview.map(os.chdir, ['D:/GitHub/Parallel/classifier']*len(c.ids))
print(dview.apply_sync(os.getcwd))

['D:\\GitHub\\Parallel\\classifier', 'D:\\GitHub\\Parallel\\classifier']


In [23]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [24]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [26]:
#sending date to engines
for i in range(len(c.ids)):
    c[i]['mnist'] = mnist

In [27]:
t = 0
s = 0
b = 0 # aggregator consumption
R = 2400

# Python optimisation variables
batch_size = 50
layer_size = [[1, 32, [5, 5], [2, 2]],
                [32, 64, [5, 5], [2, 2]],
                1000,
                10]

data_size = np.array([15000,15000])
gamma = 10
phi = 0.2
torque_aggregator = 1
stop = False

In [32]:
# Adjustment needed when change data
parameter_length_list = [32*5*5*1,32,32*64*5*5,64,1000*7*7*64,1000,1000*10,10]
w_aggregator = cnn_parameter_initial(parameter_length_list)

In [29]:
cnn = CNN(layer_size)
dview.push(dict(CNN = CNN))
#sending LinearRegression object to engines
dview['cnn'] = cnn

In [ ]:
while True:
        tic = time.time()
        snd_to_eng(w_aggregator,torque_aggregator,c)
        t_0 = t
        t = t + torque_aggregator
        dview.execute("""
import numpy as np
import time    
cnn.Rec_from_Agg(w_aggregator, torque_aggregator)
cnn.time_record()
if cnn.t > 0:
    cnn.Est_Belta(mnist)
cnn.fit(mnist)
        """)
        
        
        w_local, c_local = wc_from_eng(c,'cnn')
        w_local = flatten_matrix(w_local)
        #calculate local consumption per iteration
        c_per = np.array(c_local)/torque_aggregator
        w_aggregator = global_update(w_local, data_size)
        w_aggregator = wrangle_matrix(w_aggregator, parameter_length_list)
        if stop:
            w_final = w_aggregator
            break
        #c_local.sum() equal to c*t
        s = s + np.array(c_local).sum()+ b
        
        if t_0 > 0:
            belta_local, grad_local = bg_from_eng(c,'cnn')
            grad_local = flatten_matrix(grad_local)
            belta_aggregator = belta_update(belta_local,data_size)
            grad_aggregator = grad_update(grad_local,data_size)
            delta_aggregator = delta_update(grad_local, grad_aggregator, data_size)
            torque_aggregator, G_list = binary_search(torque_aggregator, delta_aggregator, belta_aggregator, gamma, phi)
            print('New torque is:',torque_aggregator)
        
        toc = time.time()
        b = toc - tic
        temp = s + torque_aggregator*c_per.sum() + b
        if temp >= R:
            torque_max = (R-b-s)/c_per.sum()
            G_list = np.array(G_list)
            G_min = G_list.min()
            for i, item in enumerate(G_list):
                if item >= torque_max:
                    itme = G_min
            torque_aggregator = np.argmax(G_list) + 1
            stop = True 